<a href="https://colab.research.google.com/github/F-Abir/Concept-Notes/blob/main/SVI%20VIIRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

In [2]:
!pip install xee
import xee
import xarray as xr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00


In [3]:
ee.Authenticate()
ee.Initialize(
    project = 'ee-abirnahraf',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

In [4]:
map = geemap.Map(basemap = 'SATELLITE')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
point = map.draw_last_feature.geometry()
point


In [7]:
roi = ee.FeatureCollection("FAO/GAUL/2015/level1").filterBounds(point).geometry().simplify(1000)
map.addLayer(roi, {}, 'roi')

In [8]:
evi = (
    ee.ImageCollection("NASA/VIIRS/002/VNP13A1")
    .filterDate('2015','2025')
    .select('EVI')
    .map(lambda x: x.clip(roi).copyProperties(x, ['system:time_start']))
)

evi

In [9]:
ds = xr.open_dataset(
    evi,
    engine = 'ee',
    crs = 'EPSG:4326',
    geometry = roi,
    scale = 0.01
)

In [10]:
ds1 = xr.open_dataset(
    evi,
    engine = 'ee',
    crs = 'EPSG:4326',
    geometry = roi,
    scale = 0.001
)

In [11]:
ds

<xarray.Dataset> Size: 484MB
Dimensions:  (time: 460, lon: 414, lat: 635)
Coordinates:
  * time     (time) datetime64[ns] 4kB 2015-01-01 2015-01-09 ... 2024-12-26
  * lon      (lon) float64 3kB 85.83 85.84 85.85 85.86 ... 89.94 89.95 89.96
  * lat      (lat) float64 5kB 20.87 20.88 20.89 20.9 ... 27.18 27.19 27.2 27.21
Data variables:
    EVI      (time, lon, lat) float32 484MB ...
Attributes:
    crs:      EPSG:4326

In [ ]:
ds = ds.sortby('time') * 1

In [ ]:
ds_mean = ds.mean(dim = 'time')

In [ ]:
ds_std = ds.std(dim = 'time')


In [ ]:
svi = (ds - ds_mean) / ds_std

In [ ]:
svi_annual = svi.resample(time = 'Y').mean('time')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
svi_annual.EVI.plot(
    x = 'lon',
    y = 'lat',
    col = 'time',
    col_wrap = ,
    cmap = 'turbo_r',
    robust = True
)


In [ ]:
plt.savefig('svi_annual.png', dpi = 360, bbox_inches = 'tight')



In [ ]:
!pip install netCDF4

In [ ]:
!pip install netCDF4

In [ ]:
import netCDF4

In [ ]:
svi_annual.to_netcdf('svi_annual_array.nc')

In [ ]:
svi_annual_mean = svi_annual.mean(dim = ['lon', 'lat'])

In [ ]:
svi_annual_df = svi_annual_mean.to_dataframe().dropna()

In [ ]:
svi_annual_df.to_csv('svi_annual.csv')

In [ ]:
svi_annual_df.plot()

In [ ]:
svi_mean = svi.mean(dim = ['lon', 'lat'])

In [ ]:
svi_mean_df = svi_mean.to_dataframe().dropna()

In [ ]:
svi_mean_df.to_csv('svi_16days.csv')

In [ ]:
svi_mean_df.plot()